In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from tqdm import tqdm

In [2]:
origin = '../static/datasets/original/'
file = 'train_data.csv'

In [3]:
train_data = pd.read_csv(origin + file, sep='\t')
train_data.head()

,sentence,entity,entity_tag,entity_pos_start_rel,entity_pos_end_rel,label
0,"Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...",спортсмена,PROFESSION,86,96,0
1,«За всю нашу долгую карьеру нам довелось играт...,музыкантов,PROFESSION,258,268,0
2,"Ранее, 7 декабря, толпа болельщиков перекрыла ...",Егоре Свиридове,PERSON,175,190,0
3,"В субботу, 21 июля 2018 года, на арене СК «Оли...",россиянина,NATIONALITY,122,132,0
4,Представитель талибов Забиулла Муджахид в твит...,американских захватчика,PROFESSION,109,132,-1


In [4]:
columns = ['sentence', 'label']
train_data = train_data[columns]
train_data.head()

,sentence,label
0,"Джеймс «Бадди» Макгирт (James (Buddy) McGirt, ...",0
1,«За всю нашу долгую карьеру нам довелось играт...,0
2,"Ранее, 7 декабря, толпа болельщиков перекрыла ...",0
3,"В субботу, 21 июля 2018 года, на арене СК «Оли...",0
4,Представитель талибов Забиулла Муджахид в твит...,-1


In [5]:
sentences = train_data.sentence.tolist()

In [6]:
from nltk.corpus import stopwords
ru_stopwords = set(stopwords.words("russian"))

In [7]:
import string
spec_chars = string.punctuation + '\n\xa0«»\t—…'

def remove_symbols_from_text(text, symbols):
    return "".join([ch for ch in text if ch not in symbols])

In [8]:
from nltk.stem import SnowballStemmer
import pymorphy2

morph = pymorphy2.MorphAnalyzer()

In [9]:
one_hot_words_columns = set()

In [10]:
def preprocess_sentence(sentence):
    sentence = sentence.lower()
    spec_chars = string.punctuation + '\n\t…—«»'
    sentence = remove_symbols_from_text(sentence, spec_chars)
    return sentence.split()

In [11]:
for sentence in tqdm(sentences):
    sentence = preprocess_sentence(sentence)
    for word in sentence:
        word = morph.parse(word)[0].normal_form
        if not (word in ru_stopwords or word.isnumeric()):
            one_hot_words_columns.add(word)

100%|██████████| 6637/6637 [00:19<00:00, 332.18it/s]


In [12]:
one_hot_words_columns = list(one_hot_words_columns)

In [13]:
data = [[0] * len(one_hot_words_columns)] * train_data.shape[0]

In [14]:
for i in tqdm(range(len(data))):
    sentence = preprocess_sentence(sentences[i])
    for j in range(len(data[i])):
        data[i][j] = 1 if one_hot_words_columns[j] in sentence else 0

100%|██████████| 6637/6637 [00:17<00:00, 375.08it/s]


In [15]:
df = pd.DataFrame(data, columns=one_hot_words_columns)
df.head()

,пешеход,лунгин,stones,кара,нквд,укреплять,санчёс,европарламент,микалиан,хичкок,...,сумана,амато,патаркацишвили,уильям,ракетыноситель,казнь,симмондс,калин,ссора,догонять
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
